In [1]:
import numpy as np
import pandas as pd

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Questão 1a.

In [2]:
data = pd.read_csv('womens-clothing-e-commerce-reviews.csv')

In [3]:
data.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [4]:
data = data[['Review Text','Rating']]
data = data.dropna() # remocao de valores NaN
data.head()

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [5]:
data['Rating'].value_counts()

5    12540
4     4908
3     2823
2     1549
1      821
Name: Rating, dtype: int64

In [6]:
print('Quantidade de textos: {} - Quantidade de categorias: {}'.format(data.shape[0] ,len(data['Rating'].unique())))
print('Categorias: {}'.format(data['Rating'].unique()))

Quantidade de textos: 22641 - Quantidade de categorias: 5
Categorias: [4 5 3 2 1]


In [22]:
maxlen = 300 # Cuts off reviews after 300 words
training_samples = 18112 # Trains on 18111 samples (80%)
test_samples = 2264 # Tests on 2264 samples (10%) 
validation_samples = 2264 # Validates on 2264 samples (10%)
max_words = 10000 # Considers only the top 10,000 words in the dataset

In [8]:
texts = data['Review Text'].values

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

Using TensorFlow backend.


In [10]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 14847 unique tokens.


In [11]:
data_pad = pad_sequences(sequences, maxlen=maxlen)

In [12]:
labels = data['Rating'].values
print('Shape of data tensor:', data_pad.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (22641, 250)
Shape of label tensor: (22641,)


# Splits the data into a training set and a validation set

In [13]:
indices = np.arange(data_pad.shape[0])
np.random.shuffle(indices)
data_pad = data_pad[indices]
labels = labels[indices]

In [14]:
x_train = data_pad[:training_samples]
y_train = labels[:training_samples]
x_test = data_pad[training_samples:training_samples + test_samples]
y_test = labels[training_samples:training_samples + test_samples]
x_val = data_pad[training_samples + test_samples: training_samples + test_samples + validation_samples]
y_val = labels[training_samples + test_samples: training_samples + test_samples + validation_samples]

# Questão 1b.

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM

embedding_dim = 128

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(LSTM(units=embedding_dim))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='sigmoid'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 165       
Total params: 1,415,877
Trainable params: 1,415,877
Non-trainable params: 0
_________________________________________________________________


In [17]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)
y_val = le.fit_transform(y_val)

In [18]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [19]:
lrs = [0.001, 0.01, 0.1]
epochs = [10, 20, 50]

In [23]:
from keras.optimizers import RMSprop

for lr in lrs:
    opt = RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
    model.compile(optimizer=opt,
    loss='categorical_crossentropy',
    metrics=['acc'])
    history = model.fit(x_train, y_train,
    epochs=epochs[0],
    batch_size=128,
    validation_data=(x_val, y_val))

Train on 18112 samples, validate on 2264 samples
Epoch 1/10
18112/18112 [==============================] - 72s 4ms/step - loss: 1.1407 - acc: 0.5642 - val_loss: 0.9885 - val_acc: 0.5958
Epoch 2/10
18112/18112 [==============================] - 62s 3ms/step - loss: 0.9303 - acc: 0.6149 - val_loss: 0.9272 - val_acc: 0.6215
Epoch 3/10
 8448/18112 [============>.................] - ETA: 34s - loss: 0.8687 - acc: 0.6313

KeyboardInterrupt: 